In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings; warnings.filterwarnings("ignore")

final = -1

In [2]:
film_df = pd.read_csv("/tmp/movies.csv")


title = []
tahun = []

for t in film_df.iloc[:, 1]:
    split = t.split(" ")
    if len(split) == 1:
      title.append(split[0])
      tahun.append(np.nan)
    else:
      title.append(" ".join(split[:final]))
      tahun.append(split[final:][0][1:5])

film_df["Title"] = title
film_df["Tahun"] = tahun

film_df["Tahun"].fillna("2016", inplace=True)

replacer = {"Sci-Fi":"SciFi", "Film-Noir":"Filnoir", "|":" "}

for i, t in replacer.items():
    film_df["Genre"] = film_df["Genre"].str.replace(i, t)

ind_title = lambda i : film_df.iloc[i, 1]

In [3]:
user_df = pd.read_csv("/tmp/rating.csv")

user_df.isnull().sum().sum()

0

In [4]:
user_df.iloc[:, 0].nunique()

610

In [5]:
user_df.iloc[:, 1].nunique()

9724

In [6]:
user_df.loc[:, "rating"].min()

0.5

In [7]:
user_df.loc[:, "rating"].max()

5.0

In [8]:
user_df.loc[:, "rating"].mean().round(5)

3.50156

In [9]:
df = pd.pivot_table(user_df, index=user_df.columns[1], columns=user_df.columns[0],
                    values=user_df.columns[2], fill_value=0)

df.shape

(9724, 610)

In [10]:
df.iloc[:5, :15]

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
movieId,,,,,,,,,,,,,,,
1,4,0.0,0.0,0,4,0,4.5,0,0,0.0,0,0.0,0,0,2.5
2,0,0.0,0.0,0,0,4,0.0,4,0,0.0,0,0.0,0,0,0.0
3,4,0.0,0.0,0,0,5,0.0,0,0,0.0,0,0.0,0,0,0.0
4,0,0.0,0.0,0,0,3,0.0,0,0,0.0,0,0.0,0,3,0.0
5,0,0.0,0.0,0,0,5,0.0,0,0,0.0,0,0.0,0,0,0.0


In [11]:
ind_title(10)

'American President, The'